07/11/22

Salazar Vega Rodrigo


In [1]:
import os 
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import cess_esp
from bs4 import BeautifulSoup
from pickle import load
from pickle import dump
import re
import nltk
import numpy as np

In [2]:
# Paths of importance
stopwords_path = r'C:\Users\hp\Documents\VSCode\PNL-Escom\ss\stopwords_and_lemmas\stopwords_es.txt'
lemmas_path = r'C:\Users\hp\Documents\VSCode\PNL-Escom\ss\stopwords_and_lemmas\generate.txt'
spanish_tagger = r'C:\Users\hp\Documents\VSCode\PNL-Escom\ss\stopwords_and_lemmas\spanish_tagger.pkl'

Funciones para la limpieza de texto

In [4]:
def clean_corpus(path_origin, path_destiny):
    """
        Here, you can clean your corpus, so if you can get
        a text free HTML tags and save the corpus in an unique 
        arhive 'clean_corpus.txt'.
        path_origin: the path of your original corpus.
        path_destiny: the path of yout clean corpus.
    """
    corpus = PlaintextCorpusReader(path_origin, '.*')
    file_list = corpus.fileids()
    all_text = ''
    # Get all text of the corpus
    for file in file_list:
        with open(path_origin + file, encoding = 'utf-8') as rfile:
            text = rfile.read()
            all_text += text
    # Remove HTML tags
    soup = BeautifulSoup(all_text, 'lxml')
    clean_text = soup.get_text()
    # Apply the function lower to the text
    clean_text = clean_text.lower()
    # Save the file
    with open(path_destiny + 'clean_corpus.txt', 'w', encoding = 'utf-8') as file:
        file.write(clean_text)


def normalizar(text):
    # nltk.download('stopwords') 
    '''
    Funcion para normalizar el texto y eliminar stopwords
    text : texto para normalizar
    '''
  
    from nltk.corpus import stopwords 
    stop_words = set(stopwords.words('spanish'))
    lower_string = text.lower()

    no_number_string = re.sub(r'\d+','',lower_string) 
    no_punc_string = re.sub(r'[^\w\s]','', no_number_string)  
    no_wspace_string = no_punc_string.strip() 
    # no_wspace_string 
    
    lst_string = [no_wspace_string][0].split() 
    # print(lst_string)
    no_stpwords_string="" 
    for i in lst_string: 
        if not i in stop_words: 
            no_stpwords_string += i+' '
            
    no_stpwords_string = no_stpwords_string[:-1]
    
    return no_stpwords_string


def word_tokenize(text):
    """
        Here you can tokenize yor clean corpus by words.
        text: the text you want to tokenize.
    """
    words = text.split()
    # Get only alphabetic words
    alphabetic_words = list()
    for word in words:
        token = list()
        for character in word:
            if re.match(r'^[a-záéíóúñü+$]', character):
                token.append(character)
        token = ''.join(token)
        if token != '':
            alphabetic_words.append(token)
    # Return tokens
    return alphabetic_words

# Modificada
def sentence_tokenize(text):
    """
        Here you can tokenize yor clean corpus by words.
        text: the text you want to tokenize.
    """
    tokens = nltk.data.load("tokenizers/punkt/spanish.pickle") 
    sents = tokens.tokenize(text)
    alphabetic_sents = list()
    for sent in sents:
        sent_token = word_tokenize(sent)        
        alphabetic_sents.append(sent_token)
    return alphabetic_sents

def deleteStopWords(clean_tokens):
    from nltk.corpus import stopwords
    stopwords = stopwords.words("spanish")
    
    tokens_without_stopwords = []
    for tok in clean_tokens:
        if tok not in stopwords:
            tokens_without_stopwords.append(tok)
    
    return tokens_without_stopwords
    
# Modificada
def delete_stop_words_sents(sents, path = stopwords_path):
    """
        Here you can delete the stop words from your sents.
        sents: the sents you want to clean.
        path: the path of you stopwords file.
    """
    with open(path, encoding = 'utf-8') as file:
        stop_words = file.readlines()
        stop_words = [w.strip() for w in stop_words]    
    clean_sents = list()
    for sent in sents:
        clean_sent = [word for word in sent if word not in stop_words]
        clean_sents.append(clean_sent)
    return clean_sents


def lemmatize(text, path = lemmas_path):
    """
        Here you can lemmatize your words.
        words: your words free of stop words.
        path: the path where are the lemmas you want.
    """
    lemmas = dict()
    with open(path, encoding = 'latin-1') as file:
        lines = file.readlines()
        lines = [w.strip() for w in lines]
        for line in lines:
            line = line.strip()
            if line != '':
                words = line.split()
                token = words[0].strip()
                token = token.replace('#', '')
                lemma = words[-1].strip()
                lemmas[token] = lemma
    lemmatized_text = list()
    for word in text:
        if word in lemmas.keys():
            lemmatized_text.append(lemmas[word])
        else:
            lemmatized_text.append(word)
    return lemmatized_text

# Modificada
def lemmatize_sents(sents, path = lemmas_path):
    """
        Here you can lemmatize your sents.
        words: your sents free of stop words.
        path: the path where are the lemmas you want.
    """
    lemmas = dict()
    with open(path, encoding = 'latin-1') as file:
        lines = file.readlines()
        lines = [w.strip() for w in lines]
        for line in lines:
            line = line.strip()
            if line != '':
                words = line.split()
                token = words[0].strip()
                token = token.replace('#', '')
                lemma = words[-1].strip()
                tag = words[-2].strip()
                tag = tag[0].lower()
                lemmas[(token, tag)] = (lemma, tag)
    lemmas_sents = list()
    for sent in sents:
        lemmas_sent = list()
        for word in sent:
            if word in lemmas.keys():
                lemmas_sent.append(lemmas[word])
            else:
                lemmas_sent.append(word)
        lemmas_sents.append(lemmas_sent)
    return lemmas_sents


def get_vocabulary(words):
    """
        Here you can get the vocabulary of your words.
        words: list of words.
    """
    vocabulary = list(sorted(set(words)))
    return vocabulary

# Nueva
def make_and_save_spanish_tagger(fname):

    tags_sents = list()
    for sent in cess_esp.tagged_sents():
        tags_sents_aux = [tag for (word, tag) in sent]
        tags_sents = tags_sents + tags_sents_aux

    most_used_tag_sents = nltk.FreqDist(tags_sents).max()

    default_tagger = nltk.DefaultTagger(most_used_tag_sents)

    patterns = [
        (r'.o$', 'n'),
        (r'.os$', 'n'),
        (r'.a$', 'n'),
        (r'.as$', 'n'),
        (r'.e$', 'n'),
        (r'.es$', 'n'),
        (r'.^[0-9]+$', 'z')
    ]

    regexp_tagger = nltk.RegexpTagger(patterns, backoff = default_tagger)

    cess_tagged_sents = cess_esp.tagged_sents()
    spanish_tagger = nltk.UnigramTagger(cess_tagged_sents, backoff = regexp_tagger)

    output = open(fname, 'wb')
    dump(spanish_tagger, output, -1)
    output.close() 
    
# Nueva    
def tag(sents, path = spanish_tagger):
    input_f = open(path, 'rb')
    tagger = load(input_f)
    input_f.close()
    tagged_sentences = [tagger.tag(sent) for sent in sents]
    return tagged_sentences

#### Funciones de extraccion de topicos
 

In [3]:
def splitIntoArticules(path, fname):
    with open(path + fname, 'r', encoding='utf-8') as file:
        text = file.read()
    text.replace(u'\x97', '')

    articles = re.split('<h3>', text)
    
    arts = []
    for article in articles:
        soup = BeautifulSoup(article, 'lxml')
        text = soup.get_text()
        text.replace(u'\x97', '')
        arts.append(text)
    
    return arts

Funciones para obtener la frecuencia de cada palabra con respecto al texto

In [22]:

def dict_art(artics):
    """
    Crea un diccionario donde la llave es el acronimo art más el número de articulo que es
    artics: vocabulario de articulos
    """
    art_contenidos = dict()
    for llave in range(len(artics)):
        art_contenidos['art'+str(llave+1)] = artics[llave]
    return art_contenidos

def Obtener_conteo_topico(text_norm, topicos):
    '''
    Aqui vas a obtener un diccionario de datos, donde la llave es el topico que quieres obtener en el texto y el valor es el numero de veces que aparece en el corpus
    text_norm : texto normalizado de los articulos que obtuviste
    topicos : lista que contiene palabras que a su vez son topicos
    '''
    frec_dict = dict()
    for word in text_norm:
        token = word_tokenize(word)
        for topic in topicos:
            if topic not in token:
                frec_dict[topic] = 1
            else:
                frec_dict[topic] +=1
    return frec_dict

# hecho por el breko
# def get_idf(vectors):
#     """
#         ni idea
#     """
#     num_context = len(vectors)
#     total_aparitions = [0 for i in range(num_context)]
#     for v in vectors.values():
#         i = 0
#         for element in v:
#             if element != 0:
#                 total_aparitions[i] = total_aparitions[i] + 1
#             i = i + 1
#     idf = list()
#     for element in total_aparitions:
#         if element != 0:
#             idf.append(np.log((num_context + 1) / element))
#         else:
#             idf.append(element)
#     return idf

def get_tf(articu, topicos):
    """
    Obtener valor tf de las palabras contenidas en los topicos, se obtiene su conteo y su frecuencia (numero de apariciones del topico) / total de apariciones de todos los topicos
    articu : corpus para medir
    topicos : palabras a buscar
    """
    frec_palabra= dict()

    for llave in articu.keys():
        vector = []

        for topico in range(len(topicos)):
            vector.append(articu[llave].count(topicos[topico]))

        vector = np.array(vector)
        total = np.sum(vector)
        
        if total !=0:
            vector = vector / total
        frec_palabra[llave] = vector
    
    return frec_palabra

def get_idf_topicos(art_norm, frec_dict):
    '''
    Aqui vamos a obtener el valor idf de los topicos especificos que queremos, estos topicos son previamente usados por la funcion obtener_conteo_topicos
    art_norm : texto normalizado que quieres obtener informacion
    frec_dict : diccionario de datos donde la llave es el topico del cual quieres obtener su valor idf y la el valor es el numero de apariciones que tiene en el texto normalizado
    '''
    word_idf_values = {}  
    for token in frec_dict.keys():  
        doc_containing_word = 0
        for document in art_norm:
            if token in nltk.word_tokenize(document):
                # print(token,document)
                doc_containing_word += 1
                # print(token,doc_containing_word)
        word_idf_values[token] = np.log(len(art_norm)/(1 + doc_containing_word))

def frecuencia_only_corpus (corpus, n):
    """
    valor tf*idf de las palabras que aparecen en un corpus
    corpus: todo el texto 
    corpus : texto que quieres analizar
    n : numero de palabras mas frecuentes que quieres conservar
    retorna varios valores, la lista de las palabras mas frecuentes del documento, el valor idf de las palabras, el valor tf de las palabras, el valor tf*idf y el modelo de multiplicar estas dos"""
    wordfreq = {}  
    for sentence in corpus:  
        tokens = nltk.word_tokenize(sentence)
        for token in tokens:
            if token not in wordfreq.keys():
                wordfreq[token] = 1
            else:
                wordfreq[token] += 1

    import heapq  
    most_freq = heapq.nlargest(n, wordfreq, key=wordfreq.get)  

    word_idf_values = {}  
    for token in most_freq:  
        doc_containing_word = 0
        for document in corpus:
            if token in nltk.word_tokenize(document):
                doc_containing_word += 1
        word_idf_values[token] = np.log(len(corpus)/(1 + doc_containing_word))

    word_tf_values = {}  
    for token in most_freq:  
        sent_tf_vector = []
        for document in corpus:
            doc_freq = 0
            for word in nltk.word_tokenize(document):
                if token == word:
                    doc_freq += 1
            word_tf = doc_freq/len(nltk.word_tokenize(document))
            sent_tf_vector.append(word_tf)
        word_tf_values[token] = sent_tf_vector

    tfidf_values = []  
    for token in word_tf_values.keys():  
        tfidf_sentences = []
        for tf_sentence in word_tf_values[token]:
            tf_idf_score = tf_sentence * word_idf_values[token]
            tfidf_sentences.append(tf_idf_score)
        tfidf_values.append(tfidf_sentences)

    tf_idf_model = np.asarray(tfidf_values)  
    tf_idf_model = np.transpose(tf_idf_model)  
    
    return [most_freq,word_idf_values,word_tf_values,tfidf_values,tf_idf_model]

Declaramos los topicos que después vamos a cotejar con el texto

In [6]:
topicos = ['politico', 'méxico', 'internet', 'justicia', 'presidente', 'justicia']

Obtenemos los titulos del primer archivo

In [31]:
htmlfiles = os.listdir(path='./EXCELSIOR_100_files/')


# Obtenemos y normalizamos el texto del archivo que queremos
artic = splitIntoArticules(path='./EXCELSIOR_100_files/', fname=htmlfiles[0])
art_norm = [normalizar(artic[i]) for i in range(len(artic))]

# Obtenemos el conteo de cada palabra
frec_dict = Obtener_conteo_topico(art_norm, topicos)
frec_palabra = get_tf(dict_art(art_norm), topicos)

# art_norm_tokenize = [word_tokenize(art_norm[i]) for i in range(len(art_norm))]
# art_vocabulary = [get_vocabulary(art_norm_tokenize[i]) for i in range(len(art_norm_tokenize))]
articu = dict_art(art_norm)
# frec_palabra = get_tf(articu, topicos)

In [29]:
print('\t\tTF-VALUES:')
for i in frec_palabra.keys():
    print(i)
    for j in range(len(frec_palabra[i])):
        print('\n\t',topicos[j],'=',frec_palabra[i][j], end=' ')
    print()

		TF-VALUES:
art1

	 politico = 0 
	 méxico = 0 
	 internet = 0 
	 justicia = 0 
	 presidente = 0 
	 justicia = 0 
art2

	 politico = 0.0 
	 méxico = 0.0 
	 internet = 0.0 
	 justicia = 0.3333333333333333 
	 presidente = 0.3333333333333333 
	 justicia = 0.3333333333333333 
art3

	 politico = 0.0 
	 méxico = 0.0 
	 internet = 0.0 
	 justicia = 0.0 
	 presidente = 1.0 
	 justicia = 0.0 
art4

	 politico = 0.0 
	 méxico = 0.0 
	 internet = 0.0 
	 justicia = 0.0 
	 presidente = 1.0 
	 justicia = 0.0 
art5

	 politico = 0.0 
	 méxico = 0.5 
	 internet = 0.0 
	 justicia = 0.0 
	 presidente = 0.5 
	 justicia = 0.0 
art6

	 politico = 0.0 
	 méxico = 0.6666666666666666 
	 internet = 0.0 
	 justicia = 0.0 
	 presidente = 0.3333333333333333 
	 justicia = 0.0 
art7

	 politico = 0.0 
	 méxico = 0.0 
	 internet = 0.0 
	 justicia = 0.0 
	 presidente = 1.0 
	 justicia = 0.0 
art8

	 politico = 0 
	 méxico = 0 
	 internet = 0 
	 justicia = 0 
	 presidente = 0 
	 justicia = 0 
art9

	 politico = 0.0 


Obtenemos todo el corpus